#book recommender system using collaborative clustering


In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [48]:
books=pd.read_csv("sample_data/BX-Books.csv",sep=';',error_bad_lines=False ,encoding='latin-1') 

<ipython-input-48-b3d0ab71e29e>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  books=pd.read_csv("sample_data/BX-Books.csv",sep=';',error_bad_lines=False ,encoding='latin-1')
b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 

In [49]:
books=books[['ISBN','Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher']]

In [50]:
books.rename(columns={
    "Book-Title":"title",
    "Books-Author":"author",
    "Year-Of-Publication":"year",
    "Image-URL-L":"img_url"
},inplace=True)

In [51]:
users=pd.read_csv("sample_data/BX-Users.csv",sep=';',error_bad_lines=False ,encoding='latin-1') 

<ipython-input-51-9ca8b7c8fe35>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  users=pd.read_csv("sample_data/BX-Users.csv",sep=';',error_bad_lines=False ,encoding='latin-1')


In [52]:
ratings=pd.read_csv("sample_data/BX-Book-Ratings.csv",sep=';',error_bad_lines=False ,encoding='latin-1') 

<ipython-input-52-252b9ded5838>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  ratings=pd.read_csv("sample_data/BX-Book-Ratings.csv",sep=';',error_bad_lines=False ,encoding='latin-1')


In collaborative system it is important for a user to give you ratings.

In [53]:
ratings.rename(columns={"User-ID":"user_id","Book-Rating":"rating"},inplace=True)

In [55]:
x=ratings["user_id"].value_counts()>200

In [57]:
y=x[x].index

In [63]:
ratings_with_books=ratings.merge(books,on="ISBN")

In [67]:
num_rating=ratings_with_books.groupby('title')['rating'].count().reset_index()

In [69]:
num_rating.rename(columns={"rating":"num_of_rating"},inplace=True)

In [72]:
final_rating=ratings_with_books.merge(num_rating,on ='title')

In [73]:
final_rating[[final_rating["num_of_rating"]>=50]]

KeyError: ignored